In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms, datasets
import optuna
from PIL import Image
import os
import numpy as np
import math
from torchvision.transforms import ColorJitter, Normalize


In [ ]:
low_res_folder = "./drive/MyDrive/Upscaling/buildings_low_res"
high_res_folder = "./drive/MyDrive/Upscaling/buildings"

In [ ]:



# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor
base_transform = transforms.Compose([
    transforms.ToTensor()
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)



# === Splitting into train and val sets ===

train_size = 0.8  # Proportion of data to be used for training
dataset_size = len(dataset)
split = int(train_size * dataset_size)
train_indices = list(range(split))
val_indices = list(range(split, dataset_size))

# Create train dataset as a subset of the combined dataset
train_dataset = Subset(dataset, train_indices)

# Create val dataset as a subset of the combined dataset
val_dataset = Subset(dataset, val_indices)



# === Normalization ===
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

train_dataset = [(normalize(image), normalize(target)) for image, target in train_dataset]

val_dataset = [(normalize(image), normalize(target)) for image, target in val_dataset]



# === train_data augmentation ===

# color jitter augmentation for training
train_color_jitter = ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)

# augmentation factor
augmentation_factor = 5

# augmented datasets with random color jitter
augmented_datasets = []
for _ in range(augmentation_factor):
    augmented_dataset = []
    for image, target in train_dataset:
        augmented_dataset.append((train_color_jitter(image), train_color_jitter(target)))
    augmented_datasets.append(augmented_dataset)

# combine original and augmented datasets
combined_datasets = [train_dataset] + augmented_datasets
combined_dataset = ConcatDataset(combined_datasets)



# === final data loaders ===

# Data loaders for train and val sets
batch_size = 32
train_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Number of samples in each set
print(f"Number of training samples originally: {len(train_dataset)}, now augmented to: {len(combined_dataset)}")
print(f"Number of val samples: {len(val_dataset)}")

In [ ]:
class FSRCNN(nn.Module):
    def __init__(self, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=3, padding=4, output_padding=2)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [ ]:
# Erstellen Sie das FSRCNN-Modell (wie oben definiert)
model = FSRCNN(d=56, s=12, m=4)

# Anzahl der in der Parametrsuche verwendeten Epochen
num_epochs=5

# Definieren Sie den Loss
criterion = nn.MSELoss()

# Erstellen Sie DataLoader für Ihre Trainings- und Validierungsdaten
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Definieren Sie die Optuna-Objective-Funktion
def objective(trial):
    # Vorschlagen von Hyperparametern
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    d = trial.suggest_int('d', 32, 128)
    s = trial.suggest_int('s', 5, 20)
    m = trial.suggest_int('m', 2, 8)

    # Erstellen Sie das Modell mit den vorgeschlagenen Hyperparametern
    model = FSRCNN(d=d, s=s, m=m)

    # Definieren Sie den Optimizer mit der vorgeschlagenen Lernrate
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            # Null the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Pruning
        parameters_to_prune = (
            (model.conv1, 'weight'),
            (model.conv2, 'weight'),
            (model.conv3, 'weight'),
        )
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=0.2,
        )

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        # Inform Optuna of the current loss
        trial.report(val_loss/len(val_loader), epoch)

        # Implement early stopping
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_loss/len(val_loader)

# Erstellen Sie einen Optuna-Studienobjekt und führen Sie die Optimierung durch
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Konvertieren Sie die Ergebnisse in einen DataFrame
df = study.trials_dataframe()

# Schreiben Sie den DataFrame in eine Textdatei
df.to_csv('optuna_results.txt', sep='\t', index=False)